In [12]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install scikit-learn

In [ ]:
%load_ext tensorboard

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import string

2021-07-16 13:41:59.118866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /snap/alacritty/46/usr/lib/x86_64-linux-gnu:/snap/alacritty/46/lib/x86_64-linux-gnu:/snap/alacritty/46/usr/lib/x86_64-linux-gnu/dri
2021-07-16 13:41:59.118896: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
TEST_SIZE = 1 / 3
EPOCHS = 25
VALIDATION_SPLIT = 1 / 5

VOCAB = set(string.ascii_lowercase)
MAX_LENGTH = 30

UNDEFINED = 0

In [5]:
def load_data():
    return pd.read_csv("data.csv")

def vectorize(word, vector_size):
    vector = np.array([to_categorical(ord(c) - ord('a') + 1, len(VOCAB) + 1) for c in word.lower()])
    
    vector = np.concatenate([
        [to_categorical(UNDEFINED, len(VOCAB) + 1)] * (MAX_LENGTH - len(vector)),
        vector])
    
    return vector

def prepare_X(df):
    vector_size = max(map(len, df["name"]))
    return np.array([vectorize(word, vector_size) for word in df["name"]])

In [6]:
df = load_data()
X = prepare_X(df)
y = np.array(df["gender"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE
)

In [7]:
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(MAX_LENGTH, len(VOCAB)+1)),
        keras.layers.LSTM(units=64, return_sequences=True),
        keras.layers.Dropout(0.5),
        (keras.layers.LSTM(units=64)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=1, activation=keras.activations.sigmoid),
    ]
)

2021-07-16 14:21:46.192665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /snap/alacritty/46/usr/lib/x86_64-linux-gnu:/snap/alacritty/46/lib/x86_64-linux-gnu:/snap/alacritty/46/usr/lib/x86_64-linux-gnu/dri
2021-07-16 14:21:46.192705: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-16 14:21:46.192721: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (francisthinkpad): /proc/driver/nvidia/version does not exist
2021-07-16 14:21:46.192973: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Tenso

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            23552     
_________________________________________________________________
dropout (Dropout)            (None, 30, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 56,641
Trainable params: 56,641
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    "adam",
    "bce",
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall(),
        keras.metrics.AUC()
    ]
)

In [13]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3),
    keras.callbacks.TensorBoard("logs"),
]

2021-07-16 14:28:19.282516: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-07-16 14:28:19.282591: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-07-16 14:28:19.283532: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [14]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 189203), started 0:03:46 ago. (Use '!kill 189203' to kill it.)

In [15]:
model.fit(X_train, y_train, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, callbacks=callbacks)

Epoch 1/25
   5/1584 [..............................] - ETA: 1:37 - loss: 0.4372 - binary_accuracy: 0.7937 - precision: 0.8283 - recall: 0.8367 - auc: 0.8705

2021-07-16 14:29:48.318383: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-07-16 14:29:48.318424: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-07-16 14:29:48.352340: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-07-16 14:29:48.361859: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-07-16 14:29:48.368923: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/train/plugins/profile/2021_07_16_14_29_48
2021-07-16 14:29:48.373749: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2021_07_16_14_29_48/francisthinkpad.trace.json.gz
2021-07-16 14:29:48.377674: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/train/plugins/profile/2021_07_16_14_29_48
2021-07-16 14:29:48.377819: 

1584/1584 [==============================] - 52s 33ms/step - loss: 0.3844 - binary_accuracy: 0.8308 - precision: 0.8719 - recall: 0.8601 - auc: 0.9000 - val_loss: 0.3602 - val_binary_accuracy: 0.8417 - val_precision: 0.8722 - val_recall: 0.8788 - val_auc: 0.9140
Epoch 2/25
1584/1584 [==============================] - 51s 32ms/step - loss: 0.3630 - binary_accuracy: 0.8427 - precision: 0.8815 - recall: 0.8693 - auc: 0.9109 - val_loss: 0.3387 - val_binary_accuracy: 0.8538 - val_precision: 0.8803 - val_recall: 0.8903 - val_auc: 0.9234
Epoch 3/25
1584/1584 [==============================] - 51s 32ms/step - loss: 0.3464 - binary_accuracy: 0.8516 - precision: 0.8880 - recall: 0.8770 - auc: 0.9191 - val_loss: 0.3273 - val_binary_accuracy: 0.8587 - val_precision: 0.8873 - val_recall: 0.8900 - val_auc: 0.9287
Epoch 4/25
1584/1584 [==============================] - 52s 33ms/step - loss: 0.3341 - binary_accuracy: 0.8570 - precision: 0.8921 - recall: 0.8816 - auc: 0.9247 - val_loss: 0.3185 - val_bi

KeyboardInterrupt: 

In [20]:
model.evaluate(X_test, y_test)

990/990 [==============================] - 5s 5ms/step - loss: 0.2755 - binary_accuracy: 0.8885 - precision: 0.9200 - recall: 0.9030 - auc: 0.9515


[0.27551108598709106,
 0.888524055480957,
 0.9200344085693359,
 0.9029772877693176,
 0.9515461325645447]

In [21]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [22]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/keras_metadata.pb (deflated 89%)
  adding: content/model/saved_model.pb (deflated 90%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.index (deflated 67%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 8%)
